# x402 Facilitator for Optimism 

In this notebook, we test the x402 facilitator for Optimism network.

**Deployed on Scaleway Functions:**
- Verify: `https://x402facilitatorjccmtmdr-verify.functions.fnc.fr-par.scw.cloud`
- Settle: `https://x402facilitatorjccmtmdr-settle.functions.fnc.fr-par.scw.cloud`
- Supported: `https://x402facilitatorjccmtmdr-supported.functions.fnc.fr-par.scw.cloud`

In [99]:
import requests
import json
from datetime import datetime

# Facilitator endpoint - Single function with path-based routing
# FACILITATOR_URL = "https://facilitator.fretchen.eu"
FACILITATOR_URL = "http://localhost:8080"

VERIFY_URL = f"{FACILITATOR_URL}/verify"
SETTLE_URL = f"{FACILITATOR_URL}/settle"
SUPPORTED_URL = f"{FACILITATOR_URL}/supported"


print("🚀 x402 Facilitator Endpoints (Single Function with Path Routing):")
print(f"  Base: {FACILITATOR_URL}")
print(f"  Verify: {VERIFY_URL}")
print(f"  Settle: {SETTLE_URL}")
print(f"  Supported: {SUPPORTED_URL}")

🚀 x402 Facilitator Endpoints (Single Function with Path Routing):
  Base: http://localhost:8080
  Verify: http://localhost:8080/verify
  Settle: http://localhost:8080/settle
  Supported: http://localhost:8080/supported


## Test /supported Endpoint

The `/supported` endpoint returns information about which networks, payment schemes, and tokens the facilitator supports.

**Note:** The facilitator now uses a single function with path-based routing:
- Single Scaleway Function deployment
- Path routing: `/verify`, `/settle`, `/supported`
- x402 standard compliant


## Service Fee Structure

The x402 Facilitator charges a **flat $0.01 fee per transaction** to cover:
- Gas costs for on-chain settlement (~$0.005 on Optimism)
- Infrastructure costs (hosting, monitoring)
- Nonce tracking and security

**Fee Comparison:**
- Traditional payment processors: 2.9% + $0.30 (Stripe), 2.99% + $0.49 (PayPal)
- x402 Facilitator: **Flat $0.01** regardless of transaction size

**How it works:**
1. User signs authorization for `payment_amount + $0.01` (e.g., $1.01 for a $1 payment)
2. Facilitator transfers `payment_amount` to recipient
3. Facilitator keeps the $0.01 fee to cover costs

This is much cheaper than traditional processors, especially for larger transactions!

In [100]:
# Test the /supported endpoint
response = requests.get(SUPPORTED_URL)

print(f"Status Code: {response.status_code}")
supported = response.json()
print(json.dumps(supported, indent=2))

print(f"\n✅ Supported Networks:")
for kind in supported['kinds']:
    print(f"  - {kind['network']} ({kind['scheme']} scheme)")
    if 'assets' in kind:
        for asset in kind['assets']:
            print(f"    • {asset['name']} ({asset['symbol']}): {asset['address']}")
            # Show fee information if available
            if 'fees' in asset:
                fees = asset['fees']
                print(f"      💰 Fees: {fees['feeModel']} model - ${fees['flatFeeUsd']} per transaction")
                print(f"         Minimum: ${fees['minTransactionUsd']} ({fees['minTransaction']} {fees['token']} units)")


Status Code: 200
{
  "kinds": [
    {
      "x402Version": 2,
      "scheme": "exact",
      "network": "eip155:10",
      "assets": [
        {
          "address": "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85",
          "name": "USDC",
          "symbol": "USDC",
          "decimals": 6,
          "fees": {
            "token": "USDC",
            "chainId": 10,
            "decimals": 6,
            "feeModel": "flat",
            "flatFee": "10000",
            "flatFeeUsd": 0.01,
            "minTransaction": "20000",
            "minTransactionUsd": 0.02
          }
        }
      ]
    },
    {
      "x402Version": 2,
      "scheme": "exact",
      "network": "eip155:11155420",
      "assets": [
        {
          "address": "0x5fd84259d66Cd46123540766Be93DFE6D43130D7",
          "name": "USDC",
          "symbol": "USDC",
          "decimals": 6,
          "fees": {
            "token": "USDC",
            "chainId": 11155420,
            "decimals": 6,
            "feeModel"

## Create EIP-712 Signature

In dieser Zelle kannst du zwischen **Testnet** (Optimism Sepolia) und **Mainnet** (Optimism mit echtem Geld) wechseln.

**🧪 Testnet (Standard):**
- Optimism Sepolia (Chain ID: 11155420)
- USDC: `0x5fd84259d66Cd46123540766Be93DFE6D43130D7`
- Kein echtes Geld - zum Testen gedacht

**💰 Mainnet (Echtes Geld):**
- Optimism Mainnet (Chain ID: 10)
- USDC: `0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85`
- ⚠️ **VORSICHT: Verwendet echtes USDC!**

Um zu Mainnet zu wechseln, setze `USE_MAINNET = True` in der nächsten Zelle.


In [101]:
from eth_account import Account
from eth_account.messages import encode_typed_data
from eth_keys import keys
from web3 import Web3
import os
from dotenv import load_dotenv
import secrets

# Load environment variables
load_dotenv()
# private_key = os.getenv('TEST_WALLET_PRIVATE_KEY')
private_key = os.getenv("NFT_WALLET_PRIVATE_KEY")

# Create account from private key
account = Account.from_key(private_key)
from_address = Web3.to_checksum_address(account.address)

# pay_to_address = os.getenv('NFT_WALLET_PUBLIC_KEY')
# pay_to_address = Web3.to_checksum_address(pay_to_address)

pay_to_address = "0x073f26F0C3FC100e7b075C3DC3cDE0A777497D20"

print(f"Payer Address: {from_address}")
print(f"Recipient Address: {pay_to_address}")

# ⚠️ NETWORK SELECTION - Change this to switch between testnet and mainnet
USE_MAINNET = True  # Set to True for Optimism Mainnet with REAL MONEY

if USE_MAINNET:
    CHAIN_ID = 10  # Optimism Mainnet
    USDC_ADDRESS = "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85"  # USDC on Optimism Mainnet
    USDC_NAME = "USD Coin"  # Mainnet uses "USD Coin"
    NETWORK_NAME = "Optimism Mainnet"
    CAIP2_NETWORK = "eip155:10"
    print(f"\n🚨 WARNING: Using REAL MONEY on {NETWORK_NAME}!")
else:
    CHAIN_ID = 11155420  # Optimism Sepolia
    USDC_ADDRESS = "0x5fd84259d66Cd46123540766Be93DFE6D43130D7"  # USDC on Optimism Sepolia
    USDC_NAME = "USDC"  # Testnet uses "USDC"
    NETWORK_NAME = "Optimism Sepolia (Testnet)"
    CAIP2_NETWORK = "eip155:11155420"
    print(f"\n🧪 Using testnet: {NETWORK_NAME}")

print(f"Chain ID: {CHAIN_ID}")
print(f"USDC Address: {USDC_ADDRESS}")
print(f"USDC Name: {USDC_NAME}")

# Payment configuration
payment_amount = "20000"  # $0.02 USDC (6 decimals) - amount for recipient
service_fee = "10000"       # $0.01 USDC (flat fee for facilitator)
total_amount = str(int(payment_amount) + int(service_fee))  # $0.03 USDC total

print(f"\n💰 Payment Details:")
print(f"  Payment to recipient: {payment_amount} ({int(payment_amount)/1e6:.2f} USDC)")
print(f"  Service fee: {service_fee} ({int(service_fee)/1e6:.2f} USDC)")
print(f"  Total authorization: {total_amount} ({int(total_amount)/1e6:.2f} USDC)")

# Create authorization data
valid_after = int(datetime.now().timestamp()) - 60
valid_before = int(datetime.now().timestamp()) + 3600
nonce = "0x" + secrets.token_hex(32)

# EIP-712 domain for USDC on Optimism
# CRITICAL: Mainnet uses "USD Coin", Testnet uses "USDC"!
domain_data = {
    "name": USDC_NAME,  # Network-specific name
    "version": "2",
    "chainId": CHAIN_ID,
    "verifyingContract": USDC_ADDRESS
}

# EIP-712 types for TransferWithAuthorization
types = {
    "EIP712Domain": [
        {"name": "name", "type": "string"},
        {"name": "version", "type": "string"},
        {"name": "chainId", "type": "uint256"},
        {"name": "verifyingContract", "type": "address"}
    ],
    "TransferWithAuthorization": [
        {"name": "from", "type": "address"},
        {"name": "to", "type": "address"},
        {"name": "value", "type": "uint256"},
        {"name": "validAfter", "type": "uint256"},
        {"name": "validBefore", "type": "uint256"},
        {"name": "nonce", "type": "bytes32"}
    ]
}

# Message data - IMPORTANT: Sign the TOTAL amount (payment + fee)
# The facilitator will transfer payment_amount to recipient and keep the fee
message_data = {
    "from": from_address,
    "to": pay_to_address,
    "value": int(total_amount),  # Sign total amount including fee
    "validAfter": int(valid_after),
    "validBefore": int(valid_before),
    "nonce": nonce
}

# Create the full EIP-712 message
full_message = {
    "types": types,
    "primaryType": "TransferWithAuthorization",
    "domain": domain_data,
    "message": message_data
}

# ✅ Use eth_keys for USDC-compatible signatures (manual EIP-712 hash)
# This is what USDC contract expects - raw signature over the EIP-712 hash
from eth_hash.auto import keccak

# Encode to get the EIP-712 hash components
signable_message = encode_typed_data(full_message=full_message)

# Compute the FULL EIP-712 hash according to spec:
# keccak256("\x19\x01" || domainSeparator || hashStruct(message))
prefix = bytes.fromhex("1901")
domain_separator = signable_message.header  # This is hashStruct(domain)
message_hash = signable_message.body  # This is hashStruct(message)
full_eip712_hash = keccak(prefix + domain_separator + message_hash)

# Sign using eth_keys (this creates a pure EIP-712 signature)
private_key_bytes = bytes.fromhex(private_key.replace('0x', ''))
pk = keys.PrivateKey(private_key_bytes)
signature_obj = pk.sign_msg_hash(full_eip712_hash)

# Extract v, r, s components
v_raw, r, s = signature_obj.vrs

# Adjust v to be 27 or 28 (Ethereum standard)
if v_raw == 0:
    v = 27
elif v_raw == 1:
    v = 28
else:
    v = v_raw

# Combine into signature hex string: 0x + r + s + v
r_bytes = r.to_bytes(32, byteorder='big')
s_bytes = s.to_bytes(32, byteorder='big')
signature_hex = '0x' + r_bytes.hex() + s_bytes.hex() + format(v, '02x')

print(f"\n🔐 Signature Details:")
print(f"Network: {NETWORK_NAME}")
print(f"EIP-712 Hash: {full_eip712_hash.hex()}")
print(f"Signature: {signature_hex}")
print(f"Signature length: {len(signature_hex)} chars (should be 132 with 0x)")
print(f"v: {v}, r: {r_bytes.hex()[:16]}..., s: {s_bytes.hex()[:16]}...")
print(f"Nonce: {nonce}")
print(f"\n✅ Authorized {int(total_amount)/1e6:.2f} USDC total (${int(payment_amount)/1e6:.2f} payment + ${int(service_fee)/1e6:.2f} fee)")
print(f"\n💡 Using correct domain name: '{USDC_NAME}' for {NETWORK_NAME}")

Payer Address: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C
Recipient Address: 0x073f26F0C3FC100e7b075C3DC3cDE0A777497D20

🚨 WARNING: Using REAL MONEY on Optimism Mainnet!
Chain ID: 10
USDC Address: 0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85
USDC Name: USD Coin

💰 Payment Details:
  Payment to recipient: 20000 (0.02 USDC)
  Service fee: 10000 (0.01 USDC)
  Total authorization: 30000 (0.03 USDC)

🔐 Signature Details:
Network: Optimism Mainnet
EIP-712 Hash: 1855ed7a52130ed42fec71a5a16116fb57499a2af96a4967cae4756b5d7e03ee
Signature: 0x1cd57ec9135e9de0b9e1ccf41ca77d0b197e1a0b812cdec93a0d16287efad7897866b0f705626fd06846e24e0b7b02d14665de963e89ec93d7d0eaaa4e0ededd1b
Signature length: 132 chars (should be 132 with 0x)
v: 27, r: 1cd57ec9135e9de0..., s: 7866b0f705626fd0...
Nonce: 0x180417c504b52308989598088aa0294b185fbb5bf0d1913ff4c8dc6e5f8028a4

✅ Authorized 0.03 USDC total ($0.02 payment + $0.01 fee)

💡 Using correct domain name: 'USD Coin' for Optimism Mainnet


### Debug: Prüfe ob Nonce bereits verwendet wurde

Manchmal wurde eine Nonce bereits in einem früheren Test verwendet. Lass uns prüfen:

### Debug: Vergleiche EIP-712 Domain-Parameter

Prüfe, ob die Domain-Parameter (name, version) vom Contract mit unseren Annahmen übereinstimmen:

In [102]:
# Debug: Read actual EIP-712 domain parameters from the USDC contract
print(f"🔍 Debugging EIP-712 Domain Parameters for {NETWORK_NAME}")
print(f"Contract: {USDC_ADDRESS}\n")

# Try different methods to get domain info
methods_to_try = [
    # Method 1: ERC-5267 standard (eip712Domain)
    {
        "name": "eip712Domain",
        "abi": [{
            "inputs": [],
            "name": "eip712Domain",
            "outputs": [
                {"name": "fields", "type": "bytes1"},
                {"name": "name", "type": "string"},
                {"name": "version", "type": "string"},
                {"name": "chainId", "type": "uint256"},
                {"name": "verifyingContract", "type": "address"},
                {"name": "salt", "type": "bytes32"},
                {"name": "extensions", "type": "uint256[]"}
            ],
            "stateMutability": "view",
            "type": "function"
        }]
    },
    # Method 2: Simple name() and version() getters
    {
        "name": "name",
        "abi": [{
            "inputs": [],
            "name": "name",
            "outputs": [{"name": "", "type": "string"}],
            "stateMutability": "view",
            "type": "function"
        }]
    },
    {
        "name": "version",
        "abi": [{
            "inputs": [],
            "name": "version",
            "outputs": [{"name": "", "type": "string"}],
            "stateMutability": "view",
            "type": "function"
        }]
    },
    # Method 3: DOMAIN_SEPARATOR (computed hash)
    {
        "name": "DOMAIN_SEPARATOR",
        "abi": [{
            "inputs": [],
            "name": "DOMAIN_SEPARATOR",
            "outputs": [{"name": "", "type": "bytes32"}],
            "stateMutability": "view",
            "type": "function"
        }]
    }
]

# Connect to network
if USE_MAINNET:
    rpc_url = 'https://mainnet.optimism.io'
else:
    rpc_url = 'https://sepolia.optimism.io'

w3 = Web3(Web3.HTTPProvider(rpc_url))

contract_info = {}

for method in methods_to_try:
    try:
        contract = w3.eth.contract(address=USDC_ADDRESS, abi=method["abi"])
        func = getattr(contract.functions, method["name"])
        result = func().call()
        contract_info[method["name"]] = result
        print(f"✅ {method['name']}(): {result}")
    except Exception as e:
        print(f"❌ {method['name']}(): {str(e)[:80]}...")

# Compare with our assumptions
print(f"\n📊 Comparison:")
print(f"Our assumptions:")
print(f"  name: '{domain_data['name']}'")
print(f"  version: '{domain_data['version']}'")
print(f"  chainId: {domain_data['chainId']}")
print(f"  verifyingContract: {domain_data['verifyingContract']}")

if "eip712Domain" in contract_info:
    fields, name, version, chainId, verifyingContract, salt, extensions = contract_info["eip712Domain"]
    print(f"\nContract's actual EIP-712 domain:")
    print(f"  name: '{name}'")
    print(f"  version: '{version}'")
    print(f"  chainId: {chainId}")
    print(f"  verifyingContract: {verifyingContract}")
    
    # Check for mismatches
    if name != domain_data['name']:
        print(f"\n⚠️  MISMATCH: name is '{name}' not '{domain_data['name']}'!")
    if version != domain_data['version']:
        print(f"⚠️  MISMATCH: version is '{version}' not '{domain_data['version']}'!")
    if chainId != domain_data['chainId']:
        print(f"⚠️  MISMATCH: chainId is {chainId} not {domain_data['chainId']}!")
else:
    if "name" in contract_info:
        print(f"\nContract's name: '{contract_info['name']}'")
        if contract_info['name'] != domain_data['name']:
            print(f"⚠️  MISMATCH: Expected '{domain_data['name']}'")
    
    if "version" in contract_info:
        print(f"Contract's version: '{contract_info['version']}'")
        if contract_info['version'] != domain_data['version']:
            print(f"⚠️  MISMATCH: Expected '{domain_data['version']}'")

if "DOMAIN_SEPARATOR" in contract_info:
    print(f"\n🔐 Contract's DOMAIN_SEPARATOR:")
    print(f"  {contract_info['DOMAIN_SEPARATOR'].hex()}")
    print(f"\nOur computed domain separator:")
    print(f"  {signable_message.header.hex()}")
    
    if contract_info['DOMAIN_SEPARATOR'] != signable_message.header:
        print(f"\n❌ DOMAIN_SEPARATOR MISMATCH! This explains the signature failure!")
    else:
        print(f"\n✅ Domain separators match!")

🔍 Debugging EIP-712 Domain Parameters for Optimism Mainnet
Contract: 0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85

❌ eip712Domain(): ('execution reverted', '0x')...
✅ name(): USD Coin
✅ version(): 2
✅ DOMAIN_SEPARATOR(): b'&\xd9\xc3K\xb1\xa1\xc3\x12\xf6\x9cS\xb2\xd9;\x8b\xe2\x0f\xaa\xfb\xa6:\xf2C\x8ch\x11q<\x9b\x1f\x93?'

📊 Comparison:
Our assumptions:
  name: 'USD Coin'
  version: '2'
  chainId: 10
  verifyingContract: 0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85

Contract's name: 'USD Coin'
Contract's version: '2'

🔐 Contract's DOMAIN_SEPARATOR:
  26d9c34bb1a1c312f69c53b2d93b8be20faafba63af2438c6811713c9b1f933f

Our computed domain separator:
  26d9c34bb1a1c312f69c53b2d93b8be20faafba63af2438c6811713c9b1f933f

✅ Domain separators match!


In [103]:
# Check if the current nonce was already used on-chain
print(f"Prüfe Nonce: {nonce}")
print(f"Payer: {from_address}")
print(f"Network: {NETWORK_NAME}")

# USDC contract ABI for authorizationState
usdc_abi = [
    {
        "inputs": [
            {"name": "authorizer", "type": "address"},
            {"name": "nonce", "type": "bytes32"}
        ],
        "name": "authorizationState",
        "outputs": [{"name": "", "type": "bool"}],
        "stateMutability": "view",
        "type": "function"
    }
]

# Connect to appropriate network
if USE_MAINNET:
    rpc_url = 'https://mainnet.optimism.io'
else:
    rpc_url = 'https://sepolia.optimism.io'

w3 = Web3(Web3.HTTPProvider(rpc_url))
usdc_contract = w3.eth.contract(address=USDC_ADDRESS, abi=usdc_abi)

# Check if nonce is used
nonce_used = usdc_contract.functions.authorizationState(from_address, nonce).call()

if nonce_used:
    print(f"\n❌ Diese Nonce wurde bereits verwendet!")
    print(f"Du musst die vorherige Zelle erneut ausführen, um eine neue Nonce zu generieren.")
else:
    print(f"\n✅ Diese Nonce ist noch verfügbar und kann verwendet werden.")


Prüfe Nonce: 0x180417c504b52308989598088aa0294b185fbb5bf0d1913ff4c8dc6e5f8028a4
Payer: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C
Network: Optimism Mainnet



✅ Diese Nonce ist noch verfügbar und kann verwendet werden.


In [104]:
# Now create a valid x402 payment payload with our real signature
# IMPORTANT: The 'amount' in the request is the PAYMENT amount (without fee)
# But the signature authorizes the TOTAL amount (payment + fee)

real_payment_payload = {
    "x402Version": 2,
    "resource": {
        "url": "https://api.example.com/premium-data",
        "description": "Access to premium market data",
        "mimeType": "application/json"
    },
    "accepted": {
        "scheme": "exact",
        "network": CAIP2_NETWORK,  # Dynamic: eip155:10 (mainnet) or eip155:11155420 (testnet)
        "amount": payment_amount,  # Payment amount WITHOUT fee (recipient gets this)
        "asset": USDC_ADDRESS,
        "payTo": pay_to_address,
        "maxTimeoutSeconds": 60,
        "extra": {
            "name": "USDC",
            "version": "2"
        }
    },
    "payload": {
        "signature": signature_hex,
        "authorization": {
            "from": from_address,
            "to": pay_to_address,
            "value": total_amount,  # Total amount WITH fee (what we signed)
            "validAfter": str(valid_after),
            "validBefore": str(valid_before),
            "nonce": nonce
        }
    }
}

# Create payment requirements
real_payment_requirements = {
    "scheme": "exact",
    "network": CAIP2_NETWORK,  # Dynamic network
    "amount": payment_amount,  # Payment amount WITHOUT fee
    "asset": USDC_ADDRESS,
    "payTo": pay_to_address,
    "maxTimeoutSeconds": 60,
    "extra": {
        "name": "USDC",
        "version": "2"
    }
}

# Create verify request
real_verify_request = {
    "paymentPayload": real_payment_payload,
    "paymentRequirements": real_payment_requirements
}

print(f"\n📤 Request Summary:")
print(f"  Network: {NETWORK_NAME}")
print(f"  Payment Amount (to recipient): ${int(payment_amount)/1e6:.2f} USDC")
print(f"  Authorized Amount (signed): ${int(total_amount)/1e6:.2f} USDC")
print(f"  Service Fee: ${int(service_fee)/1e6:.2f} USDC")
print(f"  Recipient: {pay_to_address}")

# Send to facilitator (deployed on Scaleway)
response = requests.post(
    VERIFY_URL,
    headers={"Content-Type": "application/json"},
    json=real_verify_request
)

result = response.json()
print(f"\n✅ Verify Response (Status {response.status_code}):")
print(json.dumps(result, indent=2))

if result.get('isValid'):
    print("\n🎉 Payment signature is valid and ready for settlement!")
    print(f"   Facilitator will transfer ${int(payment_amount)/1e6:.2f} to recipient")
    print(f"   Facilitator keeps ${int(service_fee)/1e6:.2f} as service fee")
else:
    print(f"\n❌ Validation failed: {result.get('invalidReason')}")



📤 Request Summary:
  Network: Optimism Mainnet
  Payment Amount (to recipient): $0.02 USDC
  Authorized Amount (signed): $0.03 USDC
  Service Fee: $0.01 USDC
  Recipient: 0x073f26F0C3FC100e7b075C3DC3cDE0A777497D20

  Network: Optimism Mainnet
  Payment Amount (to recipient): $0.02 USDC
  Authorized Amount (signed): $0.03 USDC
  Service Fee: $0.01 USDC
  Recipient: 0x073f26F0C3FC100e7b075C3DC3cDE0A777497D20

✅ Verify Response (Status 200):
{
  "isValid": true,
  "payer": "0xAAEBC1441323B8ad6Bdf6793A8428166b510239C"
}

🎉 Payment signature is valid and ready for settlement!
   Facilitator will transfer $0.02 to recipient
   Facilitator keeps $0.01 as service fee

✅ Verify Response (Status 200):
{
  "isValid": true,
  "payer": "0xAAEBC1441323B8ad6Bdf6793A8428166b510239C"
}

🎉 Payment signature is valid and ready for settlement!
   Facilitator will transfer $0.02 to recipient
   Facilitator keeps $0.01 as service fee


## Settlement via Facilitator

Nach erfolgreicher Verifikation kann der Facilitator die Zahlung on-chain ausführen. Der `/settle` Endpoint:
1. Führt nochmals alle Verifikationen durch
2. Ruft `transferWithAuthorization` auf dem USDC-Contract auf
3. Gibt die Transaction Hash zurück

**Voraussetzungen für Settlement:**
- Der Payer braucht USDC auf Optimism Sepolia
- Der Facilitator braucht einen Private Key (`FACILITATOR_WALLET_PRIVATE_KEY` in `.env`) und etwas ETH für Gas
- Für diesen Test kannst du deinen eigenen Private Key verwenden (im echten Betrieb sollte der Facilitator einen separaten Wallet haben)

### Prüfe Facilitator-Wallet Balance

Bevor wir Settlement versuchen, prüfen wir ob der Facilitator genug ETH für Gas hat:

In [105]:
# Check facilitator wallet balance using public key from .env
facilitator_address = os.getenv('FACILITATOR_WALLET_PUBLIC_KEY')

if facilitator_address:
    facilitator_address = Web3.to_checksum_address(facilitator_address)
    
    # Connect to appropriate network
    if USE_MAINNET:
        rpc_url = 'https://mainnet.optimism.io'
    else:
        rpc_url = 'https://sepolia.optimism.io'
    
    w3 = Web3(Web3.HTTPProvider(rpc_url))
    
    if w3.is_connected():
        balance_wei = w3.eth.get_balance(facilitator_address)
        balance_eth = w3.from_wei(balance_wei, 'ether')
        
        print(f"Facilitator Wallet: {facilitator_address}")
        print(f"Network: {NETWORK_NAME}")
        print(f"ETH Balance: {balance_eth} ETH")
        
        # Estimate gas cost (rough estimate: ~100k gas)
        estimated_gas = 100000
        gas_price = w3.eth.gas_price
        estimated_cost_wei = estimated_gas * gas_price
        estimated_cost_eth = w3.from_wei(estimated_cost_wei, 'ether')
        
        print(f"Geschätzte Gas-Kosten: ~{estimated_cost_eth:.6f} ETH")
        
        if balance_wei == 0:
            print(f"\n❌ FEHLER: Facilitator Wallet hat kein ETH!")
            print(f"Sende ETH an: {facilitator_address}")
            if USE_MAINNET:
                print(f"Bridge: https://app.optimism.io/bridge")
            else:
                print(f"Faucet: https://app.optimism.io/faucet")
        elif balance_wei < estimated_cost_wei:
            print(f"\n⚠️  WARNUNG: Balance könnte zu niedrig für Gas sein")
            print(f"Empfohlen: Mindestens {estimated_cost_eth:.6f} ETH")
        else:
            print(f"\n✅ Ausreichend ETH für Settlement vorhanden")
    else:
        print(f"❌ Konnte nicht mit {NETWORK_NAME} verbinden")
else:
    print("❌ FACILITATOR_WALLET_PUBLIC_KEY nicht in .env konfiguriert")


Facilitator Wallet: 0x3F8d2Fb6fEA24E70155bC61471936F3c9C30c206
Network: Optimism Mainnet
ETH Balance: 0.000798811469017796 ETH
Geschätzte Gas-Kosten: ~0.000000 ETH

✅ Ausreichend ETH für Settlement vorhanden


In [106]:
# Try to settle the payment (execute on-chain)
# Note: This will fail if the payer doesn't have USDC or if the facilitator wallet has no ETH for gas

print(f"\n💸 Attempting Settlement...")
print(f"  Network: {NETWORK_NAME}")
print(f"  Payment: ${int(payment_amount)/1e6:.2f} USDC → {pay_to_address}")
print(f"  Fee: ${int(service_fee)/1e6:.2f} USDC → Facilitator")
print(f"  Total: ${int(total_amount)/1e6:.2f} USDC")

if USE_MAINNET:
    print(f"\n🚨 WARNING: This will execute a REAL transaction with REAL MONEY!")
    confirm = input("Type 'YES' to confirm: ")
    if confirm != 'YES':
        print("❌ Settlement cancelled")
        raise Exception("User cancelled mainnet settlement")

settle_response = requests.post(
    SETTLE_URL,
    headers={"Content-Type": "application/json"},
    json=real_verify_request  # Same request structure as verify
)

settle_result = settle_response.json()
print(f"\n📦 Settle Response (Status {settle_response.status_code}):")
print(json.dumps(settle_result, indent=2))

if settle_result.get('success'):
    tx_hash = settle_result.get('transaction')  # API returns 'transaction', not 'transactionHash'
    print(f"\n🎉 Settlement erfolgreich!")
    print(f"\n📝 Transaction Details:")
    print(f"   Hash: {tx_hash}")
    print(f"   Recipient received: ${int(payment_amount)/1e6:.2f} USDC")
    print(f"   Service fee: ${int(service_fee)/1e6:.2f} USDC")
else:
    tx_hash = None
    print(f"\n❌ Settlement fehlgeschlagen")
    error_reason = settle_result.get('errorReason', settle_result.get('invalidReason', 'unknown'))
    print(f"   Grund: {error_reason}")
    
    if 'insufficient' in error_reason.lower():
        print(f"\n💡 Tipp: Stelle sicher, dass:")
        print(f"   - Payer hat ${int(total_amount)/1e6:.2f} USDC auf {NETWORK_NAME}")
        print(f"   - Facilitator hat ETH für Gas")


💸 Attempting Settlement...
  Network: Optimism Mainnet
  Payment: $0.02 USDC → 0x073f26F0C3FC100e7b075C3DC3cDE0A777497D20
  Fee: $0.01 USDC → Facilitator
  Total: $0.03 USDC

🚨 WARNING: This will execute a REAL transaction with REAL MONEY!

📦 Settle Response (Status 200):
{
  "success": true,
  "payer": "0xAAEBC1441323B8ad6Bdf6793A8428166b510239C",
  "transaction": "0x5e79825e13894e433c0233a6d2350ed8bcc8a9576e3eb125fa093407f31bf8e7",
  "network": "eip155:10"
}

🎉 Settlement erfolgreich!

📝 Transaction Details:
   Hash: None
   Recipient received: $0.02 USDC
   Service fee: $0.01 USDC
   Block Explorer: https://optimistic.etherscan.io/tx/None

📦 Settle Response (Status 200):
{
  "success": true,
  "payer": "0xAAEBC1441323B8ad6Bdf6793A8428166b510239C",
  "transaction": "0x5e79825e13894e433c0233a6d2350ed8bcc8a9576e3eb125fa093407f31bf8e7",
  "network": "eip155:10"
}

🎉 Settlement erfolgreich!

📝 Transaction Details:
   Hash: None
   Recipient received: $0.02 USDC
   Service fee: $0.01 USD

In [108]:
tx_hash = settle_result.get("transaction")

In [109]:
# Display transaction on block explorer
if tx_hash:
    # Block explorer URL depends on network
    if USE_MAINNET:
        explorer_url = f"https://optimistic.etherscan.io/tx/{tx_hash}"
    else:
        explorer_url = f"https://sepolia-optimism.etherscan.io/tx/{tx_hash}"
    
    print(f"🔍 Block Explorer:")
    print(f"   {explorer_url}")
    print(f"\n📊 View transaction details:")
    print(f"   • Transaction hash: {tx_hash}")
    print(f"   • Network: {NETWORK_NAME} ({CAIP2_NETWORK})")
    print(f"   • Token: USDC at {USDC_ADDRESS}")
else:
    print(f"\n⚠️  No transaction hash available")

🔍 Block Explorer:
   https://optimistic.etherscan.io/tx/0x5e79825e13894e433c0233a6d2350ed8bcc8a9576e3eb125fa093407f31bf8e7

📊 View transaction details:
   • Transaction hash: 0x5e79825e13894e433c0233a6d2350ed8bcc8a9576e3eb125fa093407f31bf8e7
   • Network: Optimism Mainnet (eip155:10)
   • Token: USDC at 0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85


In [110]:
# Check USDC balances to verify the transfer actually happened
if tx_hash:
    print(f"💰 Checking USDC Balances on {NETWORK_NAME}...\n")
    
    # USDC ERC-20 ABI for balanceOf
    erc20_abi = [
        {
            "inputs": [{"name": "account", "type": "address"}],
            "name": "balanceOf",
            "outputs": [{"name": "", "type": "uint256"}],
            "stateMutability": "view",
            "type": "function"
        }
    ]
    
    # Connect to network
    if USE_MAINNET:
        rpc_url = 'https://mainnet.optimism.io'
    else:
        rpc_url = 'https://sepolia.optimism.io'
    
    w3 = Web3(Web3.HTTPProvider(rpc_url))
    usdc_contract = w3.eth.contract(address=USDC_ADDRESS, abi=erc20_abi)
    
    # Check balances
    payer_balance = usdc_contract.functions.balanceOf(from_address).call()
    recipient_balance = usdc_contract.functions.balanceOf(pay_to_address).call()
    facilitator_balance = usdc_contract.functions.balanceOf(facilitator_address).call()
    
    print(f"📊 USDC Balances:")
    print(f"   Payer ({from_address[:10]}...): ${payer_balance/1e6:.6f} USDC")
    print(f"   Recipient ({pay_to_address[:10]}...): ${recipient_balance/1e6:.6f} USDC")
    print(f"   Facilitator ({facilitator_address[:10]}...): ${facilitator_balance/1e6:.6f} USDC")
    
    print(f"\n💡 Warum erscheint die Transaktion nicht in MetaMask Activity?")
    print(f"   • TransferWithAuthorization wird vom Facilitator ausgeführt")
    print(f"   • MetaMask zeigt nur selbst gesendete Transaktionen")
    print(f"   • Die USDC wurden trotzdem korrekt transferiert!")
    print(f"   • Prüfe den Token-Balance in MetaMask (nicht die Activity)")
    print(f"\n🔍 So siehst du den Transfer:")
    print(f"   1. Öffne Etherscan: {explorer_url}")
    print(f"   2. Klicke auf 'Logs' Tab um die Transfer Events zu sehen")
    print(f"   3. Oder gehe zu Token Transfers auf der Payer/Recipient Adresse")
else:
    print(f"\n⚠️  No transaction hash available")

💰 Checking USDC Balances on Optimism Mainnet...

📊 USDC Balances:
   Payer (0xAAEBC144...): $0.970000 USDC
   Recipient (0x073f26F0...): $10.696427 USDC
   Facilitator (0x3F8d2Fb6...): $0.000000 USDC

💡 Warum erscheint die Transaktion nicht in MetaMask Activity?
   • TransferWithAuthorization wird vom Facilitator ausgeführt
   • MetaMask zeigt nur selbst gesendete Transaktionen
   • Die USDC wurden trotzdem korrekt transferiert!
   • Prüfe den Token-Balance in MetaMask (nicht die Activity)

🔍 So siehst du den Transfer:
   1. Öffne Etherscan: https://optimistic.etherscan.io/tx/0x5e79825e13894e433c0233a6d2350ed8bcc8a9576e3eb125fa093407f31bf8e7
   2. Klicke auf 'Logs' Tab um die Transfer Events zu sehen
   3. Oder gehe zu Token Transfers auf der Payer/Recipient Adresse
📊 USDC Balances:
   Payer (0xAAEBC144...): $0.970000 USDC
   Recipient (0x073f26F0...): $10.696427 USDC
   Facilitator (0x3F8d2Fb6...): $0.000000 USDC

💡 Warum erscheint die Transaktion nicht in MetaMask Activity?
   • Tran

### 🐛 PROBLEM ENTDECKT: Fee-Implementierung ist fehlerhaft!

**Aktuelles Problem:**
Die aktuelle Implementierung hat einen fundamentalen Fehler - der Facilitator erhält keine Fee!

**Was passiert jetzt:**
1. User signiert: `from: user → to: recipient, value: $0.03`
2. Facilitator führt aus: Transfer $0.03 vom User zum Recipient
3. ❌ Recipient bekommt alle $0.03, Facilitator bekommt $0!

**Warum ist das so?**
EIP-3009 `transferWithAuthorization` erlaubt nur **einen** Transfer pro Signatur. Der signierte `to` Empfänger bekommt den gesamten Betrag.

**Die richtige Lösung:**
1. User signiert: `from: user → to: FACILITATOR, value: $0.03`
2. Facilitator führt aus: Transfer $0.03 vom User zum Facilitator
3. Facilitator macht normalen Transfer: $0.02 zum Recipient
4. ✅ Facilitator behält $0.01 als Fee

**Nächste Schritte:**
- [ ] Settlement-Code anpassen: Empfänger muss Facilitator sein
- [ ] Zweiten Transfer hinzufügen: Facilitator → Recipient
- [ ] Notebook anpassen: `to` muss Facilitator-Adresse sein, nicht Recipient
- [ ] Tests aktualisieren

Dies erklärt, warum der Facilitator $0 USDC hat!